<a href="https://colab.research.google.com/github/zideric/colab/blob/main/RNN_e_wordEmbedding_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN e Word Embedding con Keras
RNN sta per rete neurale ricorrente. sono un architettura di reti neurali che manenendo una memorria interna ci permettono di analizzare sequenze temporali di dati

costruiamo una rete neurale ricorreten per un problema di sentiment analysis utilizzando IMDB database

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense



## Scarichiamo il dataset
utilizziamo Keras per caricare imdb dataset limitandolo a 10000 parole piu comuni

In [4]:
from keras.datasets import imdb

num_words = 10000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

print("Numero di esempi nel train set: %d" % len(X_train))
print("Numero di esempi nel test set: %d" % len(X_test))


17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


Numero di esempi nel train set: 25000
Numero di esempi nel test set: 25000


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


## Processiamo i dati

per rappresentare le sequenze utilizzeremo il Word Embedding che va agggiunto come strato iniziale della rete neurale. le recensioni all'interno del corpus ovviamente possono avere una lunghezza differente.

In [5]:
longest_review = max(X_train, key=len) #applichiamo funzione max al set di day e specifichiamo che la chiave per cui vogliamo il max è len
shortest_review = min(X_train,key=len)

print("La review più lunga ha %d parole" % len(longest_review))
print("La reveiw più corta ha %d parole" % len(shortest_review))

La review più lunga ha 2494 parole
La reveiw più corta ha 11 parole


tronchiamole dopo 100 parole utilizzando la funzione pad_sequence di keras. Se una recensione ha menod di 100 parole verranno aggiunti una serie di zeri al termine per portarlo alla lungezza corretta

In [6]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100

X_train = pad_sequences(X_train, maxlen= maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

X_train.shape

(25000, 100)

## Creiamo il modello

il primo strato del modello eseguirà il Word Embedding, è un modello che ci permette di generare una serie di vettori ognuno dei quali quantifica una caratteristica delle parole. Questo tipo di rappresentazione che tiene conto di relazion e dipendenze tra le parole è un ottimo input per una RNN

La classe Embedding di Keras ha due parametri:
* il numero di parole nel nostro dizionario
* il numero di embedding vectors da generare

per aggiungere lo strato ricorrente alla nostra rete possiamo usare la classe SimpleRNN

In [8]:
from keras.layers import Embedding, SimpleRNN

model = Sequential()

model.add(Embedding(num_words, 50)) #50 è il numero di embedding vectors
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 50)          500000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                2656      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 502,689
Trainable params: 502,689
Non-trainable params: 0
_________________________________________________________________


compiliamo il modello ed eseguiamo l'addestramento su sole 10 epoche.
come algoritmo di ottimizzazione per RNN Keras consiglia di usare ** RMSprop ** anche se non ci sono evidenze sperimentali.

In [10]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=10)
model.evaluate(X_test, y_test)

Epoch 1/10
40/40 [==============================] - 5s 94ms/step - loss: 0.6869 - accuracy: 0.5353 - val_loss: 0.6063 - val_accuracy: 0.6582
Epoch 2/10
40/40 [==============================] - 3s 85ms/step - loss: 0.5225 - accuracy: 0.7639 - val_loss: 0.5010 - val_accuracy: 0.7576
Epoch 3/10
40/40 [==============================] - 3s 85ms/step - loss: 0.3701 - accuracy: 0.8501 - val_loss: 0.5365 - val_accuracy: 0.7712
Epoch 4/10
40/40 [==============================] - 3s 85ms/step - loss: 0.3003 - accuracy: 0.8862 - val_loss: 0.4369 - val_accuracy: 0.8214
Epoch 5/10
40/40 [==============================] - 3s 84ms/step - loss: 0.2266 - accuracy: 0.9197 - val_loss: 0.4111 - val_accuracy: 0.8272
Epoch 6/10
40/40 [==============================] - 3s 84ms/step - loss: 0.1748 - accuracy: 0.9417 - val_loss: 0.3957 - val_accuracy: 0.8362
Epoch 7/10
40/40 [==============================] - 3s 83ms/step - loss: 0.1254 - accuracy: 0.9630 - val_loss: 0.5219 - val_accuracy: 0.7600
Epoch 8/10
40

[0.6236118078231812, 0.801639974117279]

la rete soffre di un grande problema di overfitting. inoltre i risultati sono scarsi rispetto a un semplice percettrone multistrato.
le reti ricorrenti soffrono del problema della ** scomparsa del gradiente ** cosa che le rende inadatte per sequenze di dati troppo lunghe

in questi casi la soluzione migliore sono le ** Long Short-term memory network **